In [4]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import wavfile

from sklearn.metrics import accuracy_score

from bokeh.plotting import figure, show
from bokeh.io import output_notebook


In [5]:
from keras.layers import Conv1D, MaxPool1D, Activation, Dense, Input, Flatten
from keras.losses import binary_crossentropy
from keras.utils import Sequence
import tensorflow as tf

# Prepare Training set

In [5]:
annotations_path = '../data/MagnaTagATune/annotation_reduced.csv'
annotations = pd.read_csv(annotations_path, sep='\t')
train_set, test_set = train_test_split(annotations['mp3_path'], train_size=0.01, test_size=0.1) 

In [ ]:
x_train = []
x_test = []
for value in tqdm(train_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_train.append(data)
x_train = np.array(x_train)[:,:,np.newaxis]

for value in tqdm(test_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_test.append(data)
x_test = np.array(x_test)[:,:,np.newaxis]

with open('x_train','wb') as outfile:
    pickle.dump(x_train,outfile)
with open('x_test','wb') as outfile:
    pickle.dump(x_test,outfile)

print("X train shape:",x_train.shape)
print("X test shape:",x_test.shape)

 81%|████████  | 2097/2586 [01:02<00:14, 33.72it/s]

In [ ]:
class MagnaTagTuneSequence(Sequence)

In [ ]:
pickle.load(x_train,outfile)

In [ ]:
y_train = annotations.loc[annotations['mp3_path'].isin(train_set)].drop(columns=['mp3_path','Unnamed: 0']).values
y_test = annotations.loc[annotations['mp3_path'].isin(test_set)].drop(columns=['mp3_path','Unnamed: 0']).values
print("Y train shape:",y_train.shape)
print("Y test shape:",y_test.shape)

In [ ]:
config = tf.ConfigProto(device_count={'GPU':0})
#config.gpu_options.allow_growth = True
session = tf.Session(config=config)
keras.backend.set_session(session)

model = keras.Sequential()
model.add(Conv1D(filters=10, kernel_size=13, strides=6, input_shape=(x_train.shape[1],1)))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size=3)) #
model.add(Conv1D(filters=4, kernel_size=3, strides=1))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=y_train.shape[1], activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True,
                                         write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, 
                                         embeddings_metadata=None)]
model.fit(x_train,y_train,epochs=10, batch_size=32, callbacks=callbacks)

In [ ]:
predictions = model.predict(x_train)

In [ ]:
annotations.loc[annotations['mp3_path'].isin(test_set)].drop(columns=['mp3_path','Unnamed: 0']).columns.values[predictions[0] != 0]

In [ ]:
accuracy_score(y_train, predictions)